<a href="https://colab.research.google.com/github/TetianaMar-888/Python_for_ds_tasks/blob/main/Marinoshenko_Tetiana_HW_2_4_kNN_%D0%9A%D1%80%D0%BE%D1%81%D0%B2%D0%B0%D0%BB%D1%96%D0%B4%D0%B0%D1%86%D1%96%D1%8F_%D1%96_%D1%82%D1%8E%D0%BD%D0%B8%D0%BD%D0%B3_%D0%B3%D1%96%D0%BF%D0%B5%D1%80%D0%BF%D0%B0%D1%80%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D1%96%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [34]:
!rm -rf process_bank_churn.py* __pycache__

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import sys
sys.path.append("/content/drive/MyDrive/ColabNotebooks")

In [37]:
import sys, importlib

# Видаляємо модуль з кешу, якщо він уже був імпортований
if 'process_bank_churn' in sys.modules:
    del sys.modules['process_bank_churn']

# Імпортуємо заново
import process_bank_churn as pb
from process_bank_churn import *

In [38]:
dir(pb)

['Any',
 'Dict',
 'MinMaxScaler',
 'OneHotEncoder',
 'Tuple',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'encode_categorical_features',
 'fit_encoder',
 'fit_scaler',
 'np',
 'pd',
 'preprocess_data',
 'preprocess_new_data',
 'scale_numeric_features',
 'separate_inputs_targets',
 'split_train_val',
 'train_test_split']

In [39]:
raw_df = pd.read_csv("/content/drive/MyDrive/Machine_Learning/bank-customer-churn-prediction-dlu-course-c/train.csv")
raw_df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15779985.0,Nwankwo,678.0,France,Male,29.0,4.0,0.00,3.0,1.0,0.0,180626.36,0.0
1,1,15650086.0,Ch'in,687.0,France,Female,34.0,1.0,0.00,2.0,0.0,1.0,63736.17,0.0
2,2,15733602.0,Thompson,682.0,France,Female,52.0,6.0,0.00,3.0,0.0,0.0,179655.87,1.0
3,3,15645794.0,Macleod,753.0,Germany,Male,44.0,6.0,83347.25,2.0,1.0,0.0,161407.48,0.0
4,4,15633840.0,Hsia,544.0,Germany,Female,55.0,0.0,107747.57,1.0,1.0,0.0,176580.86,1.0


In [77]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree, export_text, DecisionTreeClassifier

In [41]:
X_train, train_targets, X_val, val_targets, input_cols, scaler, encoder = pb.preprocess_data(raw_df)

In [42]:
X_train.head()

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
7180,0.988516,0.599045,0.214286,0.1,0.626382,0.000000,1.0,1.0,0.719772,1.0,0.0,0.0,0.0,1.0
10393,0.984351,0.603819,0.375000,0.2,0.848836,0.000000,1.0,0.0,0.727603,1.0,0.0,0.0,1.0,0.0
80,0.997112,0.653938,0.303571,0.6,0.554522,0.333333,1.0,0.0,0.872180,0.0,1.0,0.0,0.0,1.0
3365,0.997533,0.568019,0.714286,0.0,0.000000,0.333333,0.0,1.0,0.257797,0.0,0.0,1.0,0.0,1.0
12236,0.987805,0.658711,0.053571,0.3,0.000000,0.333333,1.0,1.0,0.742837,1.0,0.0,0.0,0.0,1.0


1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [ ]:
!pip install mlxtend

In [90]:
import time
from mlxtend.plotting import plot_decision_regions

from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [44]:
X_train.corrwith(train_targets).to_frame(name='Exited Correlation').style.background_gradient()

,Exited Correlation
CustomerId,-0.034060
CreditScore,-0.051913
Age,0.473794
Tenure,-0.008088
Balance,0.149822
NumOfProducts,-0.285397
HasCrCard,-0.019319
IsActiveMember,-0.189398
EstimatedSalary,0.014509
Geography_France,-0.132059


In [45]:
X_raw_plot = raw_df[['Age', 'Geography']]
y_raw_plot = raw_df['Exited']

In [46]:
X = raw_df.drop(columns=["Exited"])  # усі колонки, крім цільової
y = raw_df["Exited"]                 # цільова змінна

In [47]:
print(X_train.shape)
print(train_targets.shape)

(12000, 14)
(12000,)


In [49]:
k = 5

In [50]:
new_object = [1,2,3]
train_objects = np.random.randn(100,3)

In [51]:
train_lables = np.random.choice([1,2,3], size=100)

In [52]:
train_lables

array([3, 3, 3, 1, 2, 3, 1, 2, 2, 1, 3, 2, 2, 3, 2, 3, 3, 2, 2, 1, 3, 1,
       1, 3, 1, 1, 2, 3, 3, 2, 3, 1, 1, 2, 2, 3, 1, 1, 1, 2, 1, 3, 3, 3,
       1, 3, 1, 3, 1, 3, 2, 2, 3, 3, 1, 1, 2, 1, 3, 3, 2, 2, 2, 2, 1, 2,
       1, 3, 2, 3, 3, 2, 2, 3, 3, 2, 1, 3, 3, 2, 3, 1, 2, 1, 3, 3, 1, 1,
       1, 3, 2, 3, 1, 2, 1, 3, 1, 1, 3, 1])

In [54]:
distances = [np.sqrt(np.sum((obj - new_object)**2)) for obj in train_objects]

In [55]:
top_k_indices = sorted(range(len(distances)), key=lambda i: distances[i]) [:k]

In [56]:
top_k_indices

[61, 31, 68, 60, 37]

In [57]:
np.array(distances)[top_k_indices]

array([1.40949473, 1.65280631, 1.87224069, 2.15167037, 2.39663952])

In [58]:
top_k_classes = np.array(train_lables)[top_k_indices]
top_k_classes

array([2, 1, 2, 2, 1])

In [59]:
unique, counts = np.unique(top_k_classes, return_counts=True)

In [60]:
dict(zip(unique, counts))

{np.int64(1): np.int64(2), np.int64(2): np.int64(3)}

Отже клас нового об'єкта буде 1.

In [61]:
X_train.dtypes

,0
CustomerId,float64
CreditScore,float64
Age,float64
Tenure,float64
Balance,float64
NumOfProducts,float64
HasCrCard,float64
IsActiveMember,float64
EstimatedSalary,float64
Geography_France,float64


In [62]:
X_train, train_targets, X_val, val_targets, input_cols, scaler, encoder = pb.preprocess_data(raw_df)

In [63]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, train_targets)

train_preds_proba = knn_model.predict_proba(X_train)[:, 1]
val_preds_proba = knn_model.predict_proba(X_val)[:, 1]

train_roc_auc = roc_auc_score(train_targets, train_preds_proba)
val_roc_auc = roc_auc_score(val_targets, val_preds_proba)

print(f"AUROC на тренувальних даних: {train_roc_auc:.4f}")
print(f"AUROC на валідаційних даних: {val_roc_auc:.4f}")

AUROC на тренувальних даних: 0.9559
AUROC на валідаційних даних: 0.8525


2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Псотавте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [64]:
X_train, train_targets, X_val, val_targets, input_cols, scaler, encoder = pb.preprocess_data(raw_df)

In [65]:
print(X_train.shape)
print(train_targets.shape)
print(X_val.shape)
print(val_targets.shape)

(12000, 14)
(12000,)
(3000, 14)
(3000,)


In [66]:
knn = KNeighborsClassifier()
params_knn = {'n_neighbors':np.arange(1, 25)}
knn_gs = GridSearchCV(knn, params_knn, cv=5)
knn_gs.fit(X_train, train_targets)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])})

In [67]:
knn_gs.fit(X_train, train_targets)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])})

In [68]:
knn_best = knn_gs.best_estimator_
print(knn_gs.best_params_, knn_gs.best_score_)

{'n_neighbors': np.int64(9)} 0.8670833333333334


In [69]:
knn_gs.best_estimator_

KNeighborsClassifier(n_neighbors=np.int64(9))

In [70]:
knn_best = knn_gs.best_estimator_

In [71]:
val_preds_proba = knn_best.predict_proba(X_val)

In [72]:
val_preds_proba[:4]

array([[0.88888889, 0.11111111],
       [0.88888889, 0.11111111],
       [0.88888889, 0.11111111],
       [0.77777778, 0.22222222]])

In [73]:
threshold = 0.4
val_preds = np.where(val_preds_proba[:,1]>threshold, 1,0)

In [74]:
print(classification_report(val_targets, val_preds, digits=4))

              precision    recall  f1-score   support

         0.0     0.9122    0.9176    0.9149      2390
         1.0     0.6695    0.6541    0.6617       610

    accuracy                         0.8640      3000
   macro avg     0.7908    0.7858    0.7883      3000
weighted avg     0.8629    0.8640    0.8634      3000



In [75]:
y_true = val_targets
y_pred = val_preds
preds_proba = val_preds_proba

print(classification_report(y_true, y_pred, digits=4))

roc_auc = roc_auc_score(y_true, preds_proba[:, 1])
print(f"AUROC: {roc_auc:.4f}")

              precision    recall  f1-score   support

         0.0     0.9122    0.9176    0.9149      2390
         1.0     0.6695    0.6541    0.6617       610

    accuracy                         0.8640      3000
   macro avg     0.7908    0.7858    0.7883      3000
weighted avg     0.8629    0.8640    0.8634      3000

AUROC: 0.8807


3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_state`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [78]:
dt = DecisionTreeClassifier(random_state=42)

In [79]:
param_grid = {
    'max_depth': list(range(1, 21, 2)),        # 1, 3, 5, ..., 19
    'max_leaf_nodes': list(range(2, 11, 1))    # 2, 3, ..., 10
}

In [80]:
dt_gs = GridSearchCV(
    estimator=dt,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=1
)

In [81]:
start_time = time.time()
dt_gs.fit(X_train, train_targets)
end_time = time.time()

Fitting 3 folds for each of 90 candidates, totalling 270 fits


In [82]:
print(f"\n Час виконання пошуку: {end_time - start_time:.2f} секунд")


 Час виконання пошуку: 12.19 секунд


In [83]:
print("\n Найкращі параметри Decision Tree:")
print(dt_gs.best_params_)


 Найкращі параметри Decision Tree:
{'max_depth': 5, 'max_leaf_nodes': 10}


In [84]:
dt_best = dt_gs.best_estimator_

In [89]:
train_preds_proba_dt = dt_best.predict_proba(X_train)[:, 1]
val_preds_proba_dt = dt_best.predict_proba(X_val)[:, 1]

train_roc_auc_dt = roc_auc_score(train_targets, train_preds_proba_dt)
val_roc_auc_dt = roc_auc_score(val_targets, val_preds_proba_dt)

print(f"\n Decision Tree AUROC (train): {train_roc_auc_dt:.4f}")
print(f" Decision Tree AUROC (val):   {val_roc_auc_dt:.4f}")
print(f" KNN AUROC (val): {val_roc_auc:.4f}")


 Decision Tree AUROC (train): 0.9015
 Decision Tree AUROC (val):   0.9002
 KNN AUROC (val): 0.8525


Decision Tree має кращу якість за KNN на валідації

4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [91]:
from sklearn.model_selection import RandomizedSearchCV

In [92]:
dt = DecisionTreeClassifier(random_state=42)

In [93]:
params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

In [94]:
dt_random_search = RandomizedSearchCV(
    estimator=dt,
    param_distributions=params_dt,
    n_iter=40,             # 40 випадкових комбінацій
    scoring='roc_auc',
    cv=3,
    random_state=42,       # фіксуємо випадковість для відтворюваності
    n_jobs=-1,
    verbose=1
)

In [96]:
start_time = time.time()
dt_random_search.fit(X_train, train_targets)
end_time = time.time()
print(f"\n Час виконання RandomizedSearchCV: {end_time - start_time:.2f} секунд")

Fitting 3 folds for each of 40 candidates, totalling 120 fits

 Час виконання RandomizedSearchCV: 5.27 секунд


In [97]:
print("\n Найкращі параметри Decision Tree (RandomizedSearchCV):")
print(dt_random_search.best_params_)


 Найкращі параметри Decision Tree (RandomizedSearchCV):
{'splitter': 'best', 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_leaf_nodes': np.int64(14), 'max_features': None, 'max_depth': np.int64(16), 'criterion': 'entropy'}


In [98]:
dt_random_search_best = dt_random_search.best_estimator_

In [99]:
train_preds_proba_rs = dt_random_search_best.predict_proba(X_train)[:, 1]
val_preds_proba_rs = dt_random_search_best.predict_proba(X_val)[:, 1]

train_roc_auc_rs = roc_auc_score(train_targets, train_preds_proba_rs)
val_roc_auc_rs = roc_auc_score(val_targets, val_preds_proba_rs)

print(f"\n Decision Tree (Randomized Search) AUROC (train): {train_roc_auc_rs:.4f}")
print(f" Decision Tree (Randomized Search) AUROC (val):   {val_roc_auc_rs:.4f}")


 Decision Tree (Randomized Search) AUROC (train): 0.9169
 Decision Tree (Randomized Search) AUROC (val):   0.9166


Модель показує чудову здатність класифікації

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

In [102]:
test_df = pd.read_csv("/content/drive/MyDrive/Machine_Learning/bank-customer-churn-prediction-dlu-course-c/test.csv")
test_df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15000,15594796.0,Chu,584.0,Germany,Male,30.0,2.0,146053.66,1.0,1.0,1.0,157891.86
1,15001,15642821.0,Mazzi,551.0,France,Male,39.0,5.0,0.00,2.0,1.0,1.0,67431.28
2,15002,15716284.0,Onyekachi,706.0,France,Male,43.0,8.0,0.00,2.0,1.0,0.0,156768.45
3,15003,15785078.0,Martin,717.0,Spain,Male,45.0,3.0,0.00,1.0,1.0,1.0,166909.87
4,15004,15662955.0,Kenechukwu,592.0,Spain,Male,43.0,8.0,0.00,2.0,1.0,1.0,143681.97
